In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [16]:
reddit_df = pd.read_csv('../files/reddit_mobile_operators_peru_opinions5.csv')

In [17]:
reddit_df.head()

,post_id,post_author,comment_id,comment_author,post_title,post_created_utc,link_flair_text,selftext,subreddit,upvote_ratio,comment,comment_score,comment_created_utc,operadora,mobile_operator_sentiment
0,ahmaq3,InsaneJSK,eeftoum,bestmaokaina,mejor compania internet fijo,2019-01-19 14:00:59,NaN,acabo mudarme lima quisiera saber experiencia ...,t5_2qp9h,0.73,movistar los demas hacen throttling luego de ...,3.0,2019-01-19 14:03:47,movistar,neutro
1,ahmaq3,InsaneJSK,eegc69q,morto00x,mejor compania internet fijo,2019-01-19 14:00:59,NaN,acabo mudarme lima quisiera saber experiencia ...,t5_2qp9h,0.73,depende mucho de tu zona porque el ancho de ba...,3.0,2019-01-19 17:30:52,claro,positivo
2,ahmaq3,InsaneJSK,eewh9y7,NiMierda,mejor compania internet fijo,2019-01-19 14:00:59,NaN,acabo mudarme lima quisiera saber experiencia ...,t5_2qp9h,0.73,"depende de la zona, yo tuve mala experiencia c...",2.0,2019-01-25 04:35:20,claro,positivo
3,ahmaq3,InsaneJSK,eewh9y7,NiMierda,mejor compania internet fijo,2019-01-19 14:00:59,NaN,acabo mudarme lima quisiera saber experiencia ...,t5_2qp9h,0.73,"depende de la zona, yo tuve mala experiencia c...",2.0,2019-01-25 04:35:20,movistar,positivo
4,ahmaq3,InsaneJSK,eefutn7,kambriel1,mejor compania internet fijo,2019-01-19 14:00:59,NaN,acabo mudarme lima quisiera saber experiencia ...,t5_2qp9h,0.73,"dependiendo de la zona, el mejor creo que pued...",1.0,2019-01-19 14:18:09,claro,positivo


In [19]:
# Elimina las filas con valores vacíos en la columna "comments"
reddit_df = reddit_df.dropna(subset=['comment'])

In [20]:
# minúsculas en los nombres de colores
reddit_df.columns = map(str.lower, reddit_df.columns)

In [21]:
# checkin the shape of the DF
reddit_df.shape

(3713, 15)

In [22]:
reddit_df

,post_id,post_author,comment_id,comment_author,post_title,post_created_utc,link_flair_text,selftext,subreddit,upvote_ratio,comment,comment_score,comment_created_utc,operadora,mobile_operator_sentiment
0,ahmaq3,InsaneJSK,eeftoum,bestmaokaina,mejor compania internet fijo,2019-01-19 14:00:59,NaN,acabo mudarme lima quisiera saber experiencia ...,t5_2qp9h,0.73,movistar los demas hacen throttling luego de ...,3.0,2019-01-19 14:03:47,movistar,neutro
1,ahmaq3,InsaneJSK,eegc69q,morto00x,mejor compania internet fijo,2019-01-19 14:00:59,NaN,acabo mudarme lima quisiera saber experiencia ...,t5_2qp9h,0.73,depende mucho de tu zona porque el ancho de ba...,3.0,2019-01-19 17:30:52,claro,positivo
2,ahmaq3,InsaneJSK,eewh9y7,NiMierda,mejor compania internet fijo,2019-01-19 14:00:59,NaN,acabo mudarme lima quisiera saber experiencia ...,t5_2qp9h,0.73,"depende de la zona, yo tuve mala experiencia c...",2.0,2019-01-25 04:35:20,claro,positivo
3,ahmaq3,InsaneJSK,eewh9y7,NiMierda,mejor compania internet fijo,2019-01-19 14:00:59,NaN,acabo mudarme lima quisiera saber experiencia ...,t5_2qp9h,0.73,"depende de la zona, yo tuve mala experiencia c...",2.0,2019-01-25 04:35:20,movistar,positivo
4,ahmaq3,InsaneJSK,eefutn7,kambriel1,mejor compania internet fijo,2019-01-19 14:00:59,NaN,acabo mudarme lima quisiera saber experiencia ...,t5_2qp9h,0.73,"dependiendo de la zona, el mejor creo que pued...",1.0,2019-01-19 14:18:09,claro,positivo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3713,13v5ghv,lnivin,jm7lle4,SignificantCoffee692,opiniones win internet,2023-05-29 20:23:05,opinion,recomiendan dicen garantizan servicio ahora cu...,t5_2qp9h,0.88,y darle de baja al servicio es una pesadilla.,1.0,2023-05-30 15:11:43,win,neutro
3714,13v5ghv,lnivin,jm5kgip,lnivin,opiniones win internet,2023-05-29 20:23:05,opinion,recomiendan dicen garantizan servicio ahora cu...,t5_2qp9h,0.88,"ya tuve, lo que mas me gustaba era movistartv ...",1.0,2023-05-30 02:49:02,movistar,no_definido
3715,13v5ghv,lnivin,jm5kgip,lnivin,opiniones win internet,2023-05-29 20:23:05,opinion,recomiendan dicen garantizan servicio ahora cu...,t5_2qp9h,0.88,"ya tuve, lo que mas me gustaba era movistartv ...",1.0,2023-05-30 02:49:02,win,neutro
3716,13v5ghv,lnivin,jm760vm,susananzthdev,opiniones win internet,2023-05-29 20:23:05,opinion,recomiendan dicen garantizan servicio ahora cu...,t5_2qp9h,0.88,el caso que comento es un ex trabajador de win...,1.0,2023-05-30 13:21:33,win,positivo


In [24]:
import re
reddit_df.loc[:, 'comment'] = reddit_df['comment'].apply(lambda x: x.lower())
reddit_df.loc[:, 'comment'] = reddit_df['comment'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [25]:
reddit_df

,post_id,post_author,comment_id,comment_author,post_title,post_created_utc,link_flair_text,selftext,subreddit,upvote_ratio,comment,comment_score,comment_created_utc,operadora,mobile_operator_sentiment
0,ahmaq3,InsaneJSK,eeftoum,bestmaokaina,mejor compania internet fijo,2019-01-19 14:00:59,NaN,acabo mudarme lima quisiera saber experiencia ...,t5_2qp9h,0.73,movistar los demas hacen throttling luego de ...,3.0,2019-01-19 14:03:47,movistar,neutro
1,ahmaq3,InsaneJSK,eegc69q,morto00x,mejor compania internet fijo,2019-01-19 14:00:59,NaN,acabo mudarme lima quisiera saber experiencia ...,t5_2qp9h,0.73,depende mucho de tu zona porque el ancho de ba...,3.0,2019-01-19 17:30:52,claro,positivo
2,ahmaq3,InsaneJSK,eewh9y7,NiMierda,mejor compania internet fijo,2019-01-19 14:00:59,NaN,acabo mudarme lima quisiera saber experiencia ...,t5_2qp9h,0.73,depende de la zona yo tuve mala experiencia co...,2.0,2019-01-25 04:35:20,claro,positivo
3,ahmaq3,InsaneJSK,eewh9y7,NiMierda,mejor compania internet fijo,2019-01-19 14:00:59,NaN,acabo mudarme lima quisiera saber experiencia ...,t5_2qp9h,0.73,depende de la zona yo tuve mala experiencia co...,2.0,2019-01-25 04:35:20,movistar,positivo
4,ahmaq3,InsaneJSK,eefutn7,kambriel1,mejor compania internet fijo,2019-01-19 14:00:59,NaN,acabo mudarme lima quisiera saber experiencia ...,t5_2qp9h,0.73,dependiendo de la zona el mejor creo que puede...,1.0,2019-01-19 14:18:09,claro,positivo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3713,13v5ghv,lnivin,jm7lle4,SignificantCoffee692,opiniones win internet,2023-05-29 20:23:05,opinion,recomiendan dicen garantizan servicio ahora cu...,t5_2qp9h,0.88,y darle de baja al servicio es una pesadilla,1.0,2023-05-30 15:11:43,win,neutro
3714,13v5ghv,lnivin,jm5kgip,lnivin,opiniones win internet,2023-05-29 20:23:05,opinion,recomiendan dicen garantizan servicio ahora cu...,t5_2qp9h,0.88,ya tuve lo que mas me gustaba era movistartv p...,1.0,2023-05-30 02:49:02,movistar,no_definido
3715,13v5ghv,lnivin,jm5kgip,lnivin,opiniones win internet,2023-05-29 20:23:05,opinion,recomiendan dicen garantizan servicio ahora cu...,t5_2qp9h,0.88,ya tuve lo que mas me gustaba era movistartv p...,1.0,2023-05-30 02:49:02,win,neutro
3716,13v5ghv,lnivin,jm760vm,susananzthdev,opiniones win internet,2023-05-29 20:23:05,opinion,recomiendan dicen garantizan servicio ahora cu...,t5_2qp9h,0.88,el caso que comento es un ex trabajador de win...,1.0,2023-05-30 13:21:33,win,positivo
